 # Predicting Book Success

In [26]:
#Import Dependencies
from path import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [27]:
#Import the dataset
data = Path('books_clean.csv')
df = pd.read_csv(data)

# We can see from the preview of the DataFrame that multiple variables (also called features), such as the isbn13,
#published_year, average_rating, num_pages, ratings_count, can be used to predict the outcome: whether a book will have a good 
#rating (1) or will not (0) based on the fact that an average rating below of 4.5 will not likely be successful

df.loc[df['Rating'] <= 4.0, 'Rating_Classification'] = 'Low_Rating' 
df.loc[df['Rating'] > 4.0, 'Rating_Classification'] = 'High_Rating'

df.reset_index(inplace=True, drop=True)
df.head()

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Unnamed: 10,Rating_Classification
0,9.780440e+12,4.49,7.38,870,1996446,English,Big,Famous,Top_Category,Serie,NaN,High_Rating
1,9.780320e+12,3.59,2.10,498,4367341,English,Big,Other,Other,Serie,NaN,Low_Rating
2,9.780350e+12,4.59,21.15,1728,97731,English,Big,Famous,Top_Category,Serie,NaN,High_Rating
3,9.780390e+12,3.84,2.73,464,1041597,English,Big,Other,Top_Category,Other,NaN,Low_Rating
4,9.780140e+12,4.11,5.46,532,1328143,English,Big,Other,Top_Category,Other,NaN,High_Rating


In [28]:
# We will check for the variables from all columns
df.dtypes

ISBN                     float64
Rating                   float64
Price                    float64
Pages                      int64
Rating Count               int64
Language_Dummy            object
Size_Dummy                object
Famous_Dummy              object
Categories_Dummy          object
Serie_Dummy               object
Unnamed: 10              float64
Rating_Classification     object
dtype: object

In [29]:
#Count the current NaN values from the dataframe
df.isnull().sum().sum()

1931

In [30]:
#Drop all the NaN values
df.dropna(inplace=True)

In [31]:
#Recount all the NaN values to make sure they are dropped
df.isnull().sum().sum()

0.0

In [32]:
#Use method to convert String to int
def rating(x):
    if x == 'Low_Rating':
        return 0
    if x == 'High_Rating':
        return 1
    

In [33]:
#Apply the method to the rating_classification column
df['Rating_Classification'] = df['Rating_Classification'].apply(rating)

In [34]:
#Use method to convert String to int
def binary(x):
    if x == 'Other':
        return 0
    if x == 'English'or 'Big' or 'Famous' or 'Serie' or 'Top_Category':
        return 1

In [35]:
#Apply the method to the rating_classification column
df['Size_Dummy'] = df['Size_Dummy'].apply(binary)

In [36]:
df['Serie_Dummy'] = df['Serie_Dummy'].apply(binary)

In [37]:
df['Famous_Dummy'] = df['Famous_Dummy'].apply(binary)

In [38]:
df['Language_Dummy'] = df['Language_Dummy'].apply(binary)

In [39]:
df['Categories_Dummy'] = df['Categories_Dummy'].apply(binary)

In [41]:
#Create a new dataframe with the target variables
df

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Unnamed: 10,Rating_Classification


##  Separate the Features (X) from the Target (y)

In [16]:
#The Outcome column is defined as y, or the target.
#X, or features, is created by dropping the Outcome column from the DataFrame.

y = df["Rating_Classification"]
X = df.drop(columns=["Rating_Classification","ISBN","Rating","Rating Count"])


 ## Split our data into training and testing

In [17]:
#We first split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape


(1448, 7)

In [18]:
#Examining the shape of the training set with X_train.shape returned (1636,5), meaning that there are 1636 samples (rows) and 
#five features (columns).

In [19]:
#The next step was to create a logistic regression model with the specified arguments for solver, max_iter, and random_state
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [20]:
#we trained the model with the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [21]:
#To create predictions for y-values, we used the X_test set
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.tail(20)


,Prediction,Actual
463,0,0
464,0,0
465,0,0
466,0,0
467,0,0
468,0,0
469,0,0
470,0,0
471,0,0
472,0,0


In [22]:
#The final step is to measure the accuracy of the logistic regression model created

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9855072463768116


In [23]:
#taking into account that the accuracy score is simply the percentage of predictions that are correct. In this case, 
#the model's accuracy score was 0.9834, meaning that the model was correct 98.34% of the time.

In [24]:
#  import the relevant modules for validation and print the confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,474,1
Actual 1,6,2


In [25]:
# Report of sensitivity, precission and F1. La línea 0 se aplica al predictor de diabetes como hipótesis alternativa
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       475
           1       0.67      0.25      0.36         8

    accuracy                           0.99       483
   macro avg       0.83      0.62      0.68       483
weighted avg       0.98      0.99      0.98       483

